# AICore-Bridge

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import typing
import numpy as np
import pandas as pd

In [ ]:
#| export

def timeseries_dataframe(
        adata:typing.Union[pd.DataFrame, pd.Series, dict, np.ndarray, np.recarray], 
        timezone='UTC', 
        columnnames=None):
    
    """Convert various data formats to standardized timeseries DataFrame"""

    if isinstance(adata, pd.DataFrame):
        df = adata

    elif isinstance(adata, pd.Series):
        df = pd.DataFrame(adata)

    elif isinstance(adata, dict):
        # dict/mapping of individual timeseries
        df = pd.DataFrame({
            C:pd.Series(data=A[:,1], index=pd.DatetimeIndex(A[:,0]*1e9)) if isinstance(A, np.ndarray) else A
            for C,A in adata.items()
        })

    elif adata.dtype.names is not None:
        # structured or recarray
        df = pd.DataFrame(
            data=adata.view(dtype=np.float64).reshape(adata.shape[0],len(adata.dtype))[:,range(1,len(adata.dtype))],
            index=pd.DatetimeIndex(adata.view(dtype=np.float64).reshape(adata.shape[0],len(adata.dtype))[:,0] * 1e9),
            columns=adata.dtype.names[1:]
        )

    else:
        if adata.shape[0] > 0:
            if adata.shape[1]>2:
                columns=[f"value_{str(i+1)}" for i in range(adata.shape[1]-1)] if not columnnames else [f"{str(i)}" for i in columnnames[1:]]
            else:
                columns=['value']

            df = pd.DataFrame(
                data=adata[:, 1:],
                index=pd.DatetimeIndex(adata[:,0]*1e9),
                columns=columns
            )
        else:
            return pd.DataFrame()

    df.index.name = 'time'
    if not df.index.tz:
        df.index = df.index.tz_localize('UTC').tz_convert(timezone)
    elif str(df.index.tz) != timezone:
        df.index = df.index.tz_convert(timezone)

    return df


In [ ]:
#| export
def timeseries_dataframe_from_datadict(
        data:dict, 
        recordformat='records',
        timecolumns=[]):
        
    "Convert data dict to dataframe"

    orient = recordformat.lower()
    
    if orient == 'records':
        df = pd.DataFrame.from_records(data)
        time_column = [C for C in df.columns if C.lower() in ['datetimemeasure', 'time']][0]

    elif orient == 'table':
        time_column = data['schema']['primaryKey'][0]
        df = pd.DataFrame.from_dict(data['data']).set_index(data['schema']['primaryKey'])
        df.index.name = 'time'

    df.columns = [C.lower() for C in df.columns]
    time_column = [C for C in df.columns if C in ['datetimemeasure', 'time']][0]
    df[time_column] = pd.to_datetime(df[time_column],utc=True,format='ISO8601')
    df.set_index(time_column, inplace=True)
    
    df.index.name = 'time'
    return df


In [ ]:
#| export
def timeseries_dataframe_to_datadict(
        data:typing.Union[pd.DataFrame, pd.Series, dict], 
        recordformat:str='split', 
        timezone:str='UTC', 
        reversed:bool=False):
    
    orient = recordformat.lower()

    normalized_data = self.convert_to_dataframe(data, timezone=timezone)
    normalized_data.index = normalized_data.index.map(lambda x: x.isoformat())
    
    if reversed:
        normalized_data = normalized_data[::-1]

    if orient == 'records':
        records = normalized_data.reset_index().to_dict(orient='records')
    else:
        records =  normalized_data.to_dict(orient=orient)
    

    if normalized_data.isna().any(axis=None):
        return [ {k:v for k,v in m.items() if pd.notnull(v)} for m in records]
    else:
        return records




In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()